In [597]:
import os
import warnings
import sys

import numpy as np
import pandas as pd
from mizani.formatters import percent_format
from plotnine import *
from datetime import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import norm
from IPython.core.display import HTML
from stargazer.stargazer import Stargazer
import statsmodels.nonparametric.kernel_regression as loess

warnings.filterwarnings("ignore")

print(sys.version)

3.8.9 (default, Oct 26 2021, 07:25:54) 
[Clang 13.0.0 (clang-1300.0.29.30)]


In [598]:
# Current script folder
current_path = os.getcwd()
dirname = current_path.split("da_case_studies")[0]

# location folders
data_in = dirname + "da_data_repo/cps-earnings/clean/"
data_out = dirname + "da_case_studies/ch09-gender-age-earnings/"
output = dirname + "da_case_studies/ch09-gender-age-earnings/output/"
func = dirname + "da_case_studies/ch00-tech-prep/"
sys.path.append(func)

In [599]:
data_all = pd.read_csv("/Users/user/Desktop/da_data_repo/cps-earnings/clean/morg-2014-emp.csv")

In [600]:
data_all.loc[data_all["occ2012"] == 4600, "sample"] = 1
data_all.loc[data_all["sample"].isna(), "sample"] = 0

In [601]:
data_all["sample"].value_counts()

sample
0.0    148399
1.0       917
Name: count, dtype: int64

In [602]:
def encode_marital_status(marital):
    if marital in [1, 2]:
        return 1
    else:
        return 0

In [603]:
def encode_ethnicity(ethnicity):
    if ethnicity in [1, 2, 3]:
        return 1
    else:
        return 0

In [604]:
data_all["female"] = (data_all["sex"] == 2)
data_all["w"] = data_all["earnwke"] / data_all["uhours"]
data_all["lnw"] = np.log(data_all["w"])
data_all["agesq"] = np.power(data_all["age"], 2)
data_all["married"] = data_all["marital"].apply(encode_marital_status)
data_all["mexican_american"] = data_all["ethnic"].apply(encode_ethnicity)

In [605]:
data = data_all.loc[data_all["sample"] == 1].reset_index(drop=True)

In [606]:
data.loc[:, ["earnwke", "uhours", "w"]].describe()

,earnwke,uhours,w
count,917.000000,917.000000,917.000000
mean,352.913850,30.051254,11.533942
std,274.312199,12.617595,7.164166
min,9.000000,1.000000,0.769231
25%,165.000000,20.000000,8.000000
50%,300.000000,35.000000,10.000000
75%,480.000000,40.000000,13.000000
max,2884.610000,99.000000,96.153667


In [607]:
data.loc[data["w"] >= 1, ["earnwke", "uhours", "w"]].describe()

,earnwke,uhours,w
count,916.000000,916.000000,916.000000
mean,353.288210,30.069869,11.545694
std,274.227581,12.611883,7.159231
min,9.000000,1.000000,1.000000
25%,165.000000,20.000000,8.000000
50%,300.000000,35.000000,10.000000
75%,480.000000,40.000000,13.000000
max,2884.610000,99.000000,96.153667


In [608]:
reg1 = smf.ols(formula="lnw~age+agesq", data=data).fit(cov_type="HC1")
reg2 = smf.ols(formula="lnw~female+age+agesq", data=data).fit(cov_type="HC1")
reg3 = smf.ols(formula="lnw~age+agesq+female+marital+C(ethnic)", data=data).fit(cov_type="HC1")
reg4 = smf.ols(formula="lnw~age+agesq+female+marital+C(ethnic)+age*female+female*agesq", data=data).fit(cov_type="HC1")
reg1.summary()
reg2.summary()
reg3.summary()
reg4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    lnw   R-squared:                       0.138
Model:                            OLS   Adj. R-squared:                  0.061
Method:                 Least Squares   F-statistic:                     1309.
Date:                Sat, 11 Nov 2023   Prob (F-statistic):          3.60e-143
Time:                        21:44:29   Log-Likelihood:                -111.20
No. Observations:                 159   AIC:                             250.4
Df Residuals:                     145   BIC:                             293.4
Df Model:                          13                                         
Covariance Type:                  HC1                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                3.4432      0.669      5.150      0.000       2.133       4.754
female[T.True]          -2.3267      0.796     -2.923      0.003      -3.887      -0.766
C(ethnic)[T.2.0]         0.1552      0.134      1.157      0.247      -0.108       0.418
C(ethnic)[T.3.0]        -0.4325      0.112     -3.850      0.000      -0.653      -0.212
C(ethnic)[T.4.0]         0.2151      0.176      1.225      0.221      -0.129       0.559
C(ethnic)[T.5.0]        -0.2611      0.195     -1.342      0.180      -0.643       0.120
C(ethnic)[T.6.0]         0.0618      0.118      0.522      0.601      -0.170       0.294
C(ethnic)[T.7.0]         0.0393      0.113      0.349      0.727      -0.181       0.260
C(ethnic)[T.8.0]         0.5503      0.156      3.538      0.000       0.245       0.855
age                     -0.1032      0.035     -2.984      0.003      -0.171      -0.035
age:female[T.True]       0.1600      0.043      3.737      0.000       0.076       0.244
agesq                    0.0015      0.000      3.836      0.000       0.001       0.002
female[T.True]:agesq    -0.0021      0.001     -4.189      0.000      -0.003      -0.001
marital                  0.0095      0.017      0.556      0.578      -0.024       0.043
==============================================================================
Omnibus:                       23.369   Durbin-Watson:                   1.630
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               46.614
Skew:                          -0.666   Prob(JB):                     7.55e-11
Kurtosis:                       5.294   Cond. No.                     1.92e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
[2] The condition number is large, 1.92e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [609]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from math import sqrt

In [610]:
results = []

In [611]:
def calculate_rmse(model, X, y):
    y_pred = model.predict(X)
    mse = mean_squared_error(y, y_pred)
    rmse = sqrt(mse)
    return rmse

In [612]:
results.append(("reg1", calculate_rmse(reg1, data[['age', 'agesq']], data['lnw'])))

try:
    cv_rmse_reg2 = cross_val_score(reg2, data[['female', 'age', 'agesq']], data['lnw'], scoring='neg_mean_squared_error', cv=5)
    results.append(("reg2_cv", sqrt(-cv_rmse_reg2.mean())))
except Exception as e:
    print(f"Error with reg2: {e}")

try:
    cv_rmse_reg3 = cross_val_score(reg3, data[['age', 'agesq', 'female', 'marital', 'ethnic']], data['lnw'], scoring='neg_mean_squared_error', cv=5)
    results.append(("reg3_cv", sqrt(-cv_rmse_reg3.mean())))
except Exception as e:
    print(f"Error with reg3: {e}")

try:
    cv_rmse_reg4 = cross_val_score(reg4, data[['age', 'agesq', 'female', 'marital', 'ethnic', 'age:female', 'female:agesq']], data['lnw'], scoring='neg_mean_squared_error', cv=5)
    results.append(("reg4_cv", sqrt(-cv_rmse_reg4.mean())))
except Exception as e:
    print(f"Error with reg4: {e}")

results.append(("reg1_BIC", reg1.bic))
results.append(("reg2_BIC", reg2.bic))
results.append(("reg3_BIC", reg3.bic))
results.append(("reg4_BIC", reg4.bic))

for result in results:
    print(f"{result[0]} RMSE/BIC: {result[1]}")


Error with reg2: The 'estimator' parameter of check_scoring must be an object implementing 'fit'. Got <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x12e01a310> instead.
Error with reg3: The 'estimator' parameter of check_scoring must be an object implementing 'fit'. Got <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1324d28b0> instead.
Error with reg4: "['age:female', 'female:agesq'] not in index"
reg1 RMSE/BIC: 0.4818707167391012
reg1_BIC RMSE/BIC: 1283.8309297203762
reg2_BIC RMSE/BIC: 1289.2086229575184
reg3_BIC RMSE/BIC: 284.96353945749087
reg4_BIC RMSE/BIC: 293.3705978591398
